# Installation

In [ ]:
#!pip install tabula-py[jpype]

# Imports

In [12]:
import os
import shutil
import numpy as np
import pandas as pd
from tabula import read_pdf

# Constants

In [13]:
RAW_FILE_DIRECTORY = r"..\data\pdf"

PROCESSED_FILE_DIRECTORY = r"..\data\excel"

ERROR_FILE_DIRECTORY = r"..\data\errorpdf"

data = {}

# Data Extraction

## Get Files

In [14]:
files = os.listdir(RAW_FILE_DIRECTORY)

files = [f for f in files if os.path.isfile(RAW_FILE_DIRECTORY + '/' + f)]

In [15]:
print(files)

['1 IR-O-U-0456.pdf', '10 IR-O-U-0109.pdf', '100 IR-O-U-0470.pdf', '11 IR-O-U-0500.pdf', '12 IR-O-U-0013.pdf', '13 IR-O-U-0108.pdf', '14 IR-O-U-0234.pdf', '15 IR-O-U-0120.pdf', '16 IR-O-U-0496.pdf', '17 IR-O-U-0575.pdf', '18 IR-O-U-0436.pdf', '19 IR-O-U-0490.pdf', '2 IR-O-U-0220.pdf', '20 IR-O-U-0439.pdf', '21 IR-O-U-0473.pdf', '22 IR-O-I-1441.pdf', '23 IR-O-U-0391.pdf', '24 IR-O-U-0363.pdf', '25 IR-O-U-0042.pdf', '26 IR-O-U-0570.pdf', '27 IR-O-U-0304.pdf', '28 IR-O-U-0356.pdf', '29 IR-O-U-0139.pdf', '3 IR-O-U-0306.pdf', '30 IR-O-U-0701.pdf', '31 IR-O-U-0467.pdf', '32 IR-O-U-0747.pdf', '33 IR-O-U-0273.pdf', '34 IR-O-U-0357.pdf', '35 IR-O-U-0205.pdf', '36 IR-O-U-0222.pdf', '37 IR-O-U-0323.pdf', '38 IR-O-U-0260.pdf', '39 IR-O-U-0368.pdf', '4 IR-O-I-1074.pdf', '40 IR-O-U-0020.pdf', '41 IR-O-U-0006.pdf', '42 IR-O-U-0305.pdf', '43 IR-O-I-1480.pdf', '44 IR-O-U-0447.pdf', '45 IR-O-U-0379.pdf', '46 IR-O-U-0237.pdf', '47 IR-O-U-0476.pdf', '48 IR-O-U-0378.pdf', '49 IR-O-U-0497.pdf', '5 IR-O-I-10

In [16]:
len(files)

100

In [17]:
nirf_df = read_pdf(RAW_FILE_DIRECTORY + "/" + files[0], pages='all', guess = False)

In [18]:
len(nirf_df)

17

## Read the file and export the data to Excel

In [19]:
table_first_columns = ['Academic Year',
                         '(All programs\rof all years)',
                         'Academic Year',
                         'Academic Year',
                         'Academic Year',
                         'Academic Year',
                         'Academic Year',
                         'Ph.D (Student pursuing doctoral program till 2022-23)',
                         'Financial Year',
                         'Financial Year',
                         'Financial Year',
                         #'Total no. of Funding Agencies',
                         'Financial Year',
                         'Financial Year',
                         '1. Do your institution buildings have Lifts/Ramps?',
                         '1. Has your institution implemented multiple entry/exit in your institution?'
                         'Number of faculty members entered'
                      ]

In [20]:
'''
for file in files:   
    if file.endswith('.pdf'):
        try:
            nirf_df = read_pdf(RAW_FILE_DIRECTORY + "/" + file, pages='all', guess = False)

            for df in nirf_df:
                print(df.columns[0])
        except:
            continue
'''

'\nfor file in files:   \n    if file.endswith(\'.pdf\'):\n        try:\n            nirf_df = read_pdf(RAW_FILE_DIRECTORY + "/" + file, pages=\'all\', guess = False)\n\n            for df in nirf_df:\n                print(df.columns[0])\n        except:\n            continue\n'

In [21]:
for file in files:   
    if file.endswith('.pdf'):
        try:
            nirf_df = read_pdf(RAW_FILE_DIRECTORY + "/" + file, pages='all', guess = False)

            #print(f"Extracting the data from the file {file}. The file has {len(nirf_df)} tables.")

            nirf_data = []
            table_index = 0

            for df in nirf_df:           
                if df.columns[0] == table_first_columns[table_index]:
                    if table_index != 13 and table_index != 14 and table_index != 15:
                        nirf_data.append(df)                                   
                    else:
                        data_list = [df.columns.values.tolist()]

                        for value in df.values.tolist():
                            data_list.append(value)

                        data_df = pd.DataFrame(data_list, columns=['Question', 'Answer'])

                        nirf_data.append(data_df)

                    table_index += 1
                else:
                    table_index -= 1

                    previous_df = nirf_data[table_index]

                    columns = previous_df.columns.values.tolist()

                    data_list = previous_df.values.tolist()

                    data_list.append(df.columns.values.tolist())

                    for value in df.values.tolist():
                        data_list.append(value)                

                    nirf_data[table_index] = pd.DataFrame(data_list, columns=columns)

                    table_index += 1

            #data[(file.split('-')[3]).split('.')[0]] = nirf_data       

            file_name = file.replace(".pdf", '.xlsx')

            with pd.ExcelWriter(PROCESSED_FILE_DIRECTORY + "/" + file_name) as writer:
                for index in range(len(nirf_data)):
                    nirf_data[index]['Institution'] = (file.split('.')[0])
                    nirf_data[index].to_excel(writer, sheet_name=str(index), index=False)
        except Exception as ex:
            print(f"Error while processing {file}: {ex}")
            #shutil.move(RAW_FILE_DIRECTORY + "/" + file, ERROR_FILE_DIRECTORY + "/" + file)
            continue

print("Completed the data extraction.")

Error while processing 11 IR-O-U-0500.pdf: 2 columns passed, passed data had 7 columns
Error while processing 14 IR-O-U-0234.pdf: 8 columns passed, passed data had 10 columns
Error while processing 16 IR-O-U-0496.pdf: 8 columns passed, passed data had 10 columns
Error while processing 20 IR-O-U-0439.pdf: 10 columns passed, passed data had 13 columns
Error while processing 24 IR-O-U-0363.pdf: 10 columns passed, passed data had 11 columns
Error while processing 28 IR-O-U-0356.pdf: 8 columns passed, passed data had 10 columns
Error while processing 31 IR-O-U-0467.pdf: 10 columns passed, passed data had 13 columns
Error while processing 34 IR-O-U-0357.pdf: 10 columns passed, passed data had 13 columns
Error while processing 35 IR-O-U-0205.pdf: 10 columns passed, passed data had 13 columns
Error while processing 36 IR-O-U-0222.pdf: 10 columns passed, passed data had 11 columns
Error while processing 41 IR-O-U-0006.pdf: 10 columns passed, passed data had 13 columns
Error while processing 45 